# Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

#!pip install investpy
#import investpy as inv
from time import sleep
import time

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome import service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# Criando função

In [3]:
def converter_valor(valor):
    if valor.endswith('B'):
        return float(valor.replace('B', '').replace(' ', '')) * 1_000
    elif valor.endswith('M'):
        return float(valor.replace('M', '').replace(' ', '')) * 1
    elif valor.endswith('K'):
        return float(valor.replace('K', '').replace(' ', '')) * 0.001
    elif valor == 'NA':
        return float('nan')

# Codigos

In [4]:
codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',
    #'VVBR3', 'BTOW3','ENBR3',GNDI3,IGTA3
    'BRFS3', 'BRKM5',  'CCRO3', 'CIEL3', 'CMIG4', 'COGN3', 'CPFE3',
    'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3',
     'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4',  'GOAU4', 'GOLL4', 'HAPV3',
     'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
    'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PRIO3', 'QUAL3',
    'RADL3', 'RAIL3', 'RENT3', 'SANB11', 'SBSP3','SUZB3', 'TAEE11', 'TIMS3', 'TOTS3',
    'UGPA3', 'USIM5', 'VALE3', 'VIVT3', 'WEGE3', 'WIZC3','YDUQ3','SOMA3','MDIA3'
]

# Pegando os dados do comeco

In [12]:
refazer_comeco = False

In [15]:
setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11','ITUB4','SANB11','IRBR3'}
if refazer_comeco:
    servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
    # Iniciar o navegador com o serviço
    navegador = webdriver.Chrome(service=servico)
    lista_dataframes = []
    iteracao=0
    for acao in codigos_ibovespa:
        #Comeca o tempo
        start_time = time.time()
        navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')

        # Extrair o texto de todas as datas
        elementos_data_fluxo = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_fc_3meses"]/thead/tr/th[2]/select')
        datas_fluxo = [elemento.text for elemento in elementos_data_fluxo]

        if len(datas_fluxo) == 1:
            string_de_datas = datas_fluxo[0]  # Obtenha a string que contém todas as datas
            datas_fluxo = string_de_datas.split('\n')

        # Extrair o texto de todas as datas
        elementos_data_experimental = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_fc_3meses"]/thead/tr/th[2]/select')
        datas_experimental = [elemento.text for elemento in elementos_data_experimental]

        if len(datas_experimental) == 1:
            string_de_datas = datas_experimental[0]  # Obtenha a string que contém todas as datas
            datas_experimental = string_de_datas.split('\n')
        ## Pegando valores do fluxo de caixa
        tabela_fluxo = navegador.find_element(By.ID, 'tabela_resumo_empresa_fc_3meses')
        #Pegando valores do Experimenta
        tabela_experimental = navegador.find_element(By.ID, 'pagina_empresa_experimental')
         
        if acao in setor_financeiro:
            print(f'Execucao da {acao} do setor financeiro.')
            fluxo_caixa_operacional = []
            fluxo_caixa_investimentos = []
            fluxo_caixa_financiamentos = []
            aumento_reducao_caixa_equivalentes = []
            capex_3_meses = []
            fluxo_caixa_livre_3_meses = []
            capex_12_meses = []
            fluxo_caixa_livre_12_meses = []
            lista_experimental = []

            for i_fluxo in datas_fluxo:
                
                elemento_select = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_fc_3meses"]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i_fluxo)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
                # Pegar as linhas da tabela
                linhas_fluxo = tabela_fluxo.find_elements(By.TAG_NAME, 'tr')
                lista_fluxo_caixa = []
                for linha in linhas_fluxo:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_fluxo_caixa.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())

                fluxo_caixa_operacional.append(lista_fluxo_caixa[1])
                fluxo_caixa_investimentos.append(lista_fluxo_caixa[3])
                fluxo_caixa_financiamentos.append(lista_fluxo_caixa[5])
                aumento_reducao_caixa_equivalentes.append(lista_fluxo_caixa[7])

            for i_experimental in datas_experimental:
                elemento_select = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_experimental"]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i_experimental)

                servico = webdriver.chrome.service.Service(ChromeDriverManager().install()) 
                # Pegar as linhas da tabela
                linha_expe = tabela_experimental.find_elements(By.TAG_NAME, 'tr')
                lista_experimental = []
                for linha in linha_expe:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_experimental.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())

                capex_3_meses.append(lista_experimental[1])
                fluxo_caixa_livre_3_meses.append(np.nan)
                capex_12_meses.append(lista_experimental[3])
                fluxo_caixa_livre_12_meses.append(np.nan)

        else:
            print(f'Execucao da {acao}')
            
            fluxo_caixa_operacional = []
            fluxo_caixa_investimentos = []
            fluxo_caixa_financiamentos = []
            aumento_reducao_caixa_equivalentes = []
            capex_3_meses = []
            fluxo_caixa_livre_3_meses = []
            capex_12_meses = []
            fluxo_caixa_livre_12_meses = []
            lista_experimental = []
            
            for i_fluxo in datas_fluxo:
                elemento_select = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_fc_3meses"]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i_fluxo)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
                # Pegar as linhas da tabela
                linhas_fluxo = tabela_fluxo.find_elements(By.TAG_NAME, 'tr')
                lista_fluxo_caixa = []
                for linha in linhas_fluxo:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_fluxo_caixa.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())

                fluxo_caixa_operacional.append(lista_fluxo_caixa[1])
                fluxo_caixa_investimentos.append(lista_fluxo_caixa[3])
                fluxo_caixa_financiamentos.append(lista_fluxo_caixa[5])
                aumento_reducao_caixa_equivalentes.append(lista_fluxo_caixa[7])
            
            for i_experimental in datas_experimental:
                elemento_select = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_experimental"]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i_experimental)

                servico = webdriver.chrome.service.Service(ChromeDriverManager().install()) 
                # Pegar as linhas da tabela
                linha_expe = tabela_experimental.find_elements(By.TAG_NAME, 'tr')
                lista_experimental = []
                for linha in linha_expe:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_experimental.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())

                capex_3_meses.append(lista_experimental[1])
                fluxo_caixa_livre_3_meses.append(lista_experimental[3])
                capex_12_meses.append(lista_experimental[5])
                fluxo_caixa_livre_12_meses.append(lista_experimental[7])

        data = {
            'datas':datas_fluxo,
            'fluxo_caixa_operacional': fluxo_caixa_operacional,
            'fluxo_caixa_investimentos': fluxo_caixa_investimentos,
            'fluxo_caixa_financiamentos': fluxo_caixa_financiamentos,
            'aumento_caixa_equivalentes': aumento_reducao_caixa_equivalentes,
            'capex_3_meses': capex_3_meses,
            'fluxo_caixa_livre_3_meses': fluxo_caixa_livre_3_meses,
            'capex_12_meses': capex_12_meses,
            'fluxo_caixa_livre_12_meses': fluxo_caixa_livre_12_meses
        }

        df_fluxo = pd.DataFrame(data)

        if acao in setor_financeiro:
            for col in df_fluxo.columns[(df_fluxo.columns != 'fluxo_caixa_livre_3_meses') & (df_fluxo.columns != 'fluxo_caixa_livre_12_meses') & (df_fluxo.columns != 'datas')]:
                df_fluxo[col] = df_fluxo[col].apply(converter_valor) 
        else:
            for col in df_fluxo.columns[(df_fluxo.columns != 'datas')]:
                df_fluxo[col] = df_fluxo[col].apply(converter_valor)

        df_fluxo['tic'] = acao

        lista_dataframes.append(df_fluxo)

        iteracao = iteracao+1

        if iteracao % 5 == 0:
            pd.concat(lista_dataframes).to_csv('fluxo_experimental.csv')

        end_time = time.time()

        # Calcula o tempo decorrido em segundos
        execution_time_seconds = end_time - start_time

        # Calcula o tempo decorrido em minutos
        execution_time_minutes = execution_time_seconds / 60

        # Imprime o tempo de execução em minutos
        print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")
        
pd.concat(lista_dataframes).to_csv('fluxo_experimental_2.csv')if refazer_comeco else None

# Pegando os dados da onde parou

In [33]:
data_frame_feito = pd.read_csv('fluxo_experimental_2.csv')

In [34]:
data_frame_feito.tic.unique()

array(['ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3',
       'BEEF3', 'BPAC11', 'BRAP4', 'BRFS3', 'BRKM5', 'CCRO3', 'CIEL3',
       'CMIG4', 'COGN3', 'CPFE3', 'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3'],
      dtype=object)

In [35]:
lista_dataframes = []
#setor_financeiro = ['BBAS3','BBDC3']
setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11','ITUB4','SANB11','IRBR3'}
iteracao=0
for acao in codigos_ibovespa:
    if acao in data_frame_feito['tic'].unique():
        lista_dataframes.append(data_frame_feito[data_frame_feito.tic == acao])
        print(f'Essa {acao} ja foi')
    else:
        servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
        # Iniciar o navegador com o serviço
        navegador = webdriver.Chrome(service=servico)
        #Comeca o tempo
        start_time = time.time()
        navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')

        # Extrair o texto de todas as datas
        elementos_data_fluxo = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_fc_3meses"]/thead/tr/th[2]/select')
        datas_fluxo = [elemento.text for elemento in elementos_data_fluxo]

        if len(datas_fluxo) == 1:
            string_de_datas = datas_fluxo[0]  # Obtenha a string que contém todas as datas
            datas_fluxo = string_de_datas.split('\n')

        # Extrair o texto de todas as datas
        elementos_data_experimental = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_fc_3meses"]/thead/tr/th[2]/select')
        datas_experimental = [elemento.text for elemento in elementos_data_experimental]

        if len(datas_experimental) == 1:
            string_de_datas = datas_experimental[0]  # Obtenha a string que contém todas as datas
            datas_experimental = string_de_datas.split('\n')
        ## Pegando valores do fluxo de caixa
        tabela_fluxo = navegador.find_element(By.ID, 'tabela_resumo_empresa_fc_3meses')
        #Pegando valores do Experimenta
        tabela_experimental = navegador.find_element(By.ID, 'pagina_empresa_experimental')
        
        if acao in setor_financeiro:
            print(f'Execucao da {acao} do setor financeiro.')
            fluxo_caixa_operacional = []
            fluxo_caixa_investimentos = []
            fluxo_caixa_financiamentos = []
            aumento_reducao_caixa_equivalentes = []
            capex_3_meses = []
            fluxo_caixa_livre_3_meses = []
            capex_12_meses = []
            fluxo_caixa_livre_12_meses = []
            lista_experimental = []

            for i_fluxo in datas_fluxo:
                
                elemento_select = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_fc_3meses"]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i_fluxo)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
                # Pegar as linhas da tabela
                linhas_fluxo = tabela_fluxo.find_elements(By.TAG_NAME, 'tr')
                lista_fluxo_caixa = []
                for linha in linhas_fluxo:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_fluxo_caixa.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())

                fluxo_caixa_operacional.append(lista_fluxo_caixa[1])
                fluxo_caixa_investimentos.append(lista_fluxo_caixa[3])
                fluxo_caixa_financiamentos.append(lista_fluxo_caixa[5])
                aumento_reducao_caixa_equivalentes.append(lista_fluxo_caixa[7])

            for i_experimental in datas_experimental:
                elemento_select = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_experimental"]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i_experimental)

                servico = webdriver.chrome.service.Service(ChromeDriverManager().install()) 
                # Pegar as linhas da tabela
                linha_expe = tabela_experimental.find_elements(By.TAG_NAME, 'tr')
                lista_experimental = []
                for linha in linha_expe:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_experimental.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())

                capex_3_meses.append(lista_experimental[1])
                fluxo_caixa_livre_3_meses.append(np.nan)
                capex_12_meses.append(lista_experimental[3])
                fluxo_caixa_livre_12_meses.append(np.nan)

        else:
            print(f'Execucao da {acao}')
            
            fluxo_caixa_operacional = []
            fluxo_caixa_investimentos = []
            fluxo_caixa_financiamentos = []
            aumento_reducao_caixa_equivalentes = []
            capex_3_meses = []
            fluxo_caixa_livre_3_meses = []
            capex_12_meses = []
            fluxo_caixa_livre_12_meses = []
            lista_experimental = []
            
            for i_fluxo in datas_fluxo:
                elemento_select = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_fc_3meses"]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i_fluxo)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
                # Pegar as linhas da tabela
                linhas_fluxo = tabela_fluxo.find_elements(By.TAG_NAME, 'tr')
                lista_fluxo_caixa = []
                for linha in linhas_fluxo:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_fluxo_caixa.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())

                fluxo_caixa_operacional.append(lista_fluxo_caixa[1])
                fluxo_caixa_investimentos.append(lista_fluxo_caixa[3])
                fluxo_caixa_financiamentos.append(lista_fluxo_caixa[5])
                aumento_reducao_caixa_equivalentes.append(lista_fluxo_caixa[7])
            
            for i_experimental in datas_experimental:
                elemento_select = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_experimental"]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i_experimental)

                servico = webdriver.chrome.service.Service(ChromeDriverManager().install()) 
                # Pegar as linhas da tabela
                linha_expe = tabela_experimental.find_elements(By.TAG_NAME, 'tr')
                lista_experimental = []
                for linha in linha_expe:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_experimental.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())

                capex_3_meses.append(lista_experimental[1])
                fluxo_caixa_livre_3_meses.append(lista_experimental[3])
                capex_12_meses.append(lista_experimental[5])
                fluxo_caixa_livre_12_meses.append(lista_experimental[7])

        data = {
            'datas':datas_fluxo,
            'fluxo_caixa_operacional': fluxo_caixa_operacional,
            'fluxo_caixa_investimentos': fluxo_caixa_investimentos,
            'fluxo_caixa_financiamentos': fluxo_caixa_financiamentos,
            'aumento_caixa_equivalentes': aumento_reducao_caixa_equivalentes,
            'capex_3_meses': capex_3_meses,
            'fluxo_caixa_livre_3_meses': fluxo_caixa_livre_3_meses,
            'capex_12_meses': capex_12_meses,
            'fluxo_caixa_livre_12_meses': fluxo_caixa_livre_12_meses
        }

        df_fluxo = pd.DataFrame(data)

        if acao in setor_financeiro:
            for col in df_fluxo.columns[(df_fluxo.columns != 'fluxo_caixa_livre_3_meses') & (df_fluxo.columns != 'fluxo_caixa_livre_12_meses') & (df_fluxo.columns != 'datas')]:
                df_fluxo[col] = df_fluxo[col].apply(converter_valor) 
        else:
            for col in df_fluxo.columns[(df_fluxo.columns != 'datas')]:
                df_fluxo[col] = df_fluxo[col].apply(converter_valor)

        df_fluxo['tic'] = acao

        lista_dataframes.append(df_fluxo)

        iteracao = iteracao+1

        if iteracao % 5 == 0:
            pd.concat(lista_dataframes).to_csv('fluxo_experimental.csv')

        end_time = time.time()

        # Calcula o tempo decorrido em segundos
        execution_time_seconds = end_time - start_time

        # Calcula o tempo decorrido em minutos
        execution_time_minutes = execution_time_seconds / 60

        # Imprime o tempo de execução em minutos
        print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")

pd.concat(lista_dataframes).to_csv('fluxo_experimental_2.csv')

Essa ABEV3 ja foi
Essa AZUL4 ja foi
Essa B3SA3 ja foi
Essa BBAS3 ja foi
Essa BBDC3 ja foi
Essa BBDC4 ja foi
Essa BBSE3 ja foi
Essa BEEF3 ja foi
Essa BPAC11 ja foi
Essa BRAP4 ja foi
Essa BRFS3 ja foi
Essa BRKM5 ja foi
Essa CCRO3 ja foi
Essa CIEL3 ja foi
Essa CMIG4 ja foi
Essa COGN3 ja foi
Essa CPFE3 ja foi
Essa CRFB3 ja foi
Essa CSAN3 ja foi
Essa CSNA3 ja foi
Essa CVCB3 ja foi
Execucao da CYRE3
Tempo de execução da ação CYRE3: 3.91 minutos
Execucao da ECOR3
Tempo de execução da ação ECOR3: 4.02 minutos
Execucao da EGIE3
Tempo de execução da ação EGIE3: 4.02 minutos
Execucao da ELET3
Tempo de execução da ação ELET3: 3.97 minutos
Execucao da ELET6
Tempo de execução da ação ELET6: 3.85 minutos
Execucao da EMBR3
Tempo de execução da ação EMBR3: 3.72 minutos
Execucao da ENGI11
Tempo de execução da ação ENGI11: 3.73 minutos
Execucao da EQTL3
Tempo de execução da ação EQTL3: 4.12 minutos
Execucao da EZTC3
Tempo de execução da ação EZTC3: 4.06 minutos
Execucao da FLRY3
Tempo de execução da ação

In [32]:
pd.concat(lista_dataframes).iloc[:,2:].to_csv('fluxo_experimental_2.csv')